In [1]:
from tool import Tool

import ollama
import openai
import json

In [2]:
openai.base_url = "http://localhost:11434/v1"
openai.api_key = 'ollama'

client = ollama.Client()

In [3]:
### FUNCTION DEFINITION ###

import ast

def add_numbers(numbers: list[float]) -> float:
    if isinstance(numbers, str):
        numbers = ast.literal_eval(numbers)
    return sum(map(float, numbers))

In [4]:
tools = {
    'add_numbers': Tool(
        definition={
            'type': 'function',
            'function': {
                'name': 'add_numbers',
                'description': 'Adds the values of all numbers in the list `numbers`',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'numbers': {
                            'type': 'list[float]',
                            'description': 'A list of real numbers',
                        },
                    },
                    'required': ['numbers'],
                },
            }},
        function=add_numbers
    ),
}


def tool_defs(): return [x.definition for x in tools.values()]

In [14]:
def generate_tool_outputs(tool_calls):
    tool_outputs = []

    for tool_call in tool_calls:
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']

        if function_name in tools:
            output = tools[function_name].call(**arguments)
            print(f'Tool output ({function_name}{arguments}): {output}')
            tool_outputs.append({
                'role': 'tool',
                'name': function_name,
                # 'content': str(output),
                'content': json.dumps(f"{function_name}({arguments}): {str(output)}")
            })

    return tool_outputs


In [22]:
def run_assistant(model, message):
    chat_messages = [
        {
            "role": "user",
            "content": message,
        }
    ]

    res = client.chat(
        model=model,
        messages=chat_messages,
        tools=tool_defs(),
    )

    print(json.dumps(res, indent=2))

    msg = res['message']

    # Check if the model called tools
    if msg.get('tool_calls'):
        print(msg['tool_calls'])
        tool_outputs = generate_tool_outputs(msg['tool_calls'])
        # chat_messages.append(msg)
        chat_messages.extend(tool_outputs)

        for chunk in client.chat(
            model=model,
            messages=chat_messages,
            stream=True
        ):
            print(chunk['message']['content'], end='', flush=True)

    else:
        print(msg['content'])

In [27]:
# run_assistant("llama3.1-function-calling", "What's 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 999 + 19241284176?")
# run_assistant("llama3.1-function-calling", "Hi, what's up!")

run_assistant("llama3.2", "What's 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 999 + 19241284176?")
# run_assistant("llama3.2", "Hi, what's up!")

{
  "model": "llama3.2",
  "created_at": "2024-09-27T15:37:00.872391909Z",
  "message": {
    "role": "assistant",
    "content": "",
    "tool_calls": [
      {
        "function": {
          "name": "add_numbers",
          "arguments": {
            "numbers": "[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 999, 19241284176]"
          }
        }
      }
    ]
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 1038078525,
  "load_duration": 18255032,
  "prompt_eval_count": 206,
  "prompt_eval_duration": 104286000,
  "eval_count": 54,
  "eval_duration": 873342000
}
[{'function': {'name': 'add_numbers', 'arguments': {'numbers': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 999, 19241284176]'}}}]
Tool output (add_numbers{'numbers': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 999, 19241284176]'}): 19241285230.0
This code calculates the sum of a list of numbers that includes two large integers.

The first argument to `add_numbers()` is a dictionary with one key-value pair: `'numbers'` maps to a list of intege